### Deep Learning
#### Compute the solution to the model when you use a neural network with three layers and 25 nodes per layer on capital and a three-point finite approximation to $z_t$ using Tauchen’s method. 

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import norm
import matplotlib.pyplot as plt


In [3]:
class Params:
    def __init__(self):
        # Technology parameters
        self.alpha = 0.33  # Capital share
        self.beta = 0.97   # Time discount factor
        self.delta = 0.1   # Depreciation rate
        self.rho = 1 / self.beta - 1  # Time discount rate

        # Productivity shock parameters
        self.rho_z = 0.95  # Persistence of productivity shocks
        self.sigma_e = 0.007  # Standard deviation of productivity shocks

        # Utility parameters
        self.psi = 1.0  # Inverse Frisch elasticity
        self.sigma = 1.0  # CRRA parameter


In [4]:
# Production function with productivity shock
def prod_fn(k, l, z, params):
    return np.exp(z) * (k ** params.alpha) * (l ** (1 - params.alpha))

# Marginal product of capital
def rtn_to_capital(k, l, z, params):
    return np.exp(z) * params.alpha * (k ** (params.alpha - 1)) * (l ** (1 - params.alpha))

# Marginal utility of consumption
def u_prime(c, params):
    return c ** (-params.sigma)


In [5]:
class SteadyState:
    def __init__(self, params):
        self.params = params
        self.z_ss = 0 
        self.solve_steady_state()

    def solve_steady_state(self):
        """
        Compute the steady-state values for capital, labor, consumption, and output.
        """
        p = self.params
        z = self.z_ss

        y_to_k = (p.delta + p.rho) / p.alpha
        k_to_l = y_to_k ** (1 / (p.alpha - 1))
        css = (k_to_l ** p.alpha - p.delta * k_to_l) * ((1 - p.alpha) * (k_to_l ** p.alpha)) ** (1 / p.psi)
        css = css ** (1 / (1 + p.sigma / p.psi))
        lss = ((1 - p.alpha) * (k_to_l ** p.alpha) * css ** (-p.sigma)) ** (1 / p.psi)
        kss = k_to_l * lss
        yss = y_to_k * kss

        self.k_ss = kss
        self.l_ss = lss
        self.c_ss = css
        self.y_ss = yss

        
    def get_steady_state(self):
        """
        Return the steady-state values as a dictionary.
        """
        return {
            "k_ss": self.k_ss,
            "l_ss": self.l_ss,
            "c_ss": self.c_ss,
            "y_ss": self.y_ss,
        }

# Initialize parameters
params = Params()

# Solve for the steady state
steady_state = SteadyState(params)
ss_values = steady_state.get_steady_state()

# Print steady-state results
print("\nSteady State variables: \n")
for key, value in ss_values.items():
    print(f"{key}: {value:.4f}")



Steady State variables: 

k_ss: 3.7612
l_ss: 0.9465
c_ss: 1.1161
y_ss: 1.4923


In [6]:
# Tauchen's Method for Discretizing Productivity Shocks
def tauchen(rho, sigma, m=3, n=7):
    """
    Tauchen's method to discretize the AR(1) process:
        z_t = rho * z_{t-1} + sigma * epsilon_t
    where epsilon_t ~ N(0, 1).

    Parameters:
    - rho: AR(1) persistence parameter
    - sigma: Standard deviation of the shocks
    - m: Number of standard deviations to cover in the grid
    - n: Number of grid points

    Returns:
    - z: Grid for the state variable
    - pi: Transition probability matrix
    """
    # Standard deviation of the stationary distribution
    std_z = sigma / np.sqrt(1 - rho**2)

    # Create evenly spaced grid for z
    z_max = m * std_z
    z_min = -z_max
    z = np.linspace(z_min, z_max, n)
    step = (z_max - z_min) / (n - 1)

    # Transition probability matrix
    pi = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if j == 0:
                pi[i, j] = norm.cdf((z[0] - rho * z[i] + step / 2) / sigma)
            elif j == n - 1:
                pi[i, j] = 1 - norm.cdf((z[j] - rho * z[i] - step / 2) / sigma)
            else:
                pi[i, j] = (
                    norm.cdf((z[j] - rho * z[i] + step / 2) / sigma)
                    - norm.cdf((z[j] - rho * z[i] - step / 2) / sigma)
                )
    return z, pi

# Parameters for Tauchen's
m = 3             # Number of standard deviations to cover
num_z = 7         # Number of grid points for z

# Discretize productivity shocks
z_grid, z_prob = tauchen(params.rho_z, params.sigma_e, m=m, n=num_z)

In [23]:
# Capital grid
cover_grid = 0.25        # Fractional coverage around the steady-state
k_min = steady_state.k_ss * (1 - cover_grid)
k_max = steady_state.k_ss * (1 + cover_grid)
num_k = 1001             # Number of grid points for k
k_grid = np.linspace(k_min, k_max, num_k)
